## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [1]:
import json
import os
import chi

from chi.server import *
from chi.lease import *
from chi.network import *

from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'
VERBOSE=False  #Set to true for extra output

#Config with your project and site
chi.use_site('CHI@UC')
chi.set('project_name', 'CH-822154')

#Insert keypair name
key_name = 'mac_chameleon_key'  # Change to your keypair

#GENI Pem file
geni_pem_file='/home/mwhicks2/work/geni-mwhick01.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")+'12'
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type=""
server_count=1

#Network Config
physical_network='exogeni'
cidr='192.168.42.0/24'

uc_allocation_start='192.168.42.101'
uc_allocation_end='192.168.42.150'
uc_gateway='192.168.42.100'
tacc_allocation_start='192.168.42.201'
tacc_allocation_end='192.168.42.250'
tacc_gateway='192.168.42.200'

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


## Create Chicago Network and Server

#### Create a Lease at Chicago

In [2]:
#Set the region
chi.use_site('CHI@UC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


{'created_at': '2021-04-28 15:57:17',
 'updated_at': '2021-04-28 15:57:24',
 'id': 'fe94192b-6394-4fc8-aa55-6b0017feaa0b',
 'name': 'mwhicks212Lease',
 'user_id': '956cc2ceba48aa3093c77c40c5aff9c16c0ac5c54919989b4d0ac43e64762096',
 'project_id': 'ae76673270164b048b59d3bd30676721',
 'start_date': '2021-04-28T15:58:00.000000',
 'end_date': '2021-04-29T15:57:00.000000',
 'trust_id': 'cc147a25c6f348abb51b82883dbbe96c',
 'status': 'PENDING',
 'degraded': False,
 'reservations': [{'created_at': '2021-04-28 15:57:23',
   'updated_at': '2021-04-28 15:57:24',
   'id': '6cc568b8-dc41-4f42-8bb4-d2bd9ff7e2cf',
   'lease_id': 'fe94192b-6394-4fc8-aa55-6b0017feaa0b',
   'resource_id': '9a291b92-a502-4d64-954d-e88077030dc8',
   'resource_type': 'virtual:floatingip',
   'status': 'pending',
   'missing_resources': False,
   'resources_changed': False,
   'network_id': '44b38c44-2a42-4b6d-b129-6c8f1b2a1375',
   'amount': 1,
   'required_floatingips': []},
  {'created_at': '2021-04-28 15:57:17',
   'upda

In [3]:
#Get the lease by name
uc_lease = get_lease(get_lease_id(lease_name))
    
#Print the lease info
if VERBOSE:
    print(json.dumps(uc_lease, indent=2))
else:
    print('uc_lease: ' + uc_lease['name'] + ', ' + uc_lease['id'])

uc_lease: mwhicks212Lease, fe94192b-6394-4fc8-aa55-6b0017feaa0b


#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [4]:
#Get the lease by name
uc_lease = get_lease(get_lease_id(lease_name))

uc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', uc_lease['reservations']))[0]['id']
uc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', uc_lease['reservations']))[0]['id']
uc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', uc_lease['reservations']))[0]['id']

print("uc_compute_reservation_id: " + uc_compute_reservation_id)
print("uc_network_reservation_id: " + uc_network_reservation_id)
print("uc_floatingip_reservation_id: " + uc_floatingip_reservation_id)

uc_compute_reservation_id: 7f78fda7-fb88-4826-9042-46056aa823ba
uc_network_reservation_id: db9e6aa9-fbea-40b2-94bb-05cd3de30f30
uc_floatingip_reservation_id: 6cc568b8-dc41-4f42-8bb4-d2bd9ff7e2cf


#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [6]:
chi.use_site('CHI@UC')    # Optional, defaults to 'CHI@UC'

#Get the network
uc_network = get_network(get_network_id(network_name))
uc_network_id = uc_network['id']
uc_network_vlan = uc_network['provider:segmentation_id']

if VERBOSE:
    print(json.dumps(uc_network, indent=2))
else:
    print('uc_network: ' + uc_network['name'] + ', ' + uc_network['id'])
    print('uc segmenation ID/VLAN: ' + str(uc_network_vlan))

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org
uc_network: mwhicks212Net, ea9236b1-032a-4990-8837-8e30f00cbebb
uc segmenation ID/VLAN: 3295


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [7]:
subnet = create_subnet(subnet_name, 
                       uc_network_id, 
                       cidr=cidr,
                      allocation_pool_start=uc_allocation_start,
                      allocation_pool_end=uc_allocation_end,
                      gateway_ip=uc_gateway)



#print(json.dumps(subnet, indent=2))

In [8]:
if VERBOSE:
    print(json.dumps(subnet, indent=2))
else:
    print('Subnet: ' + subnet['name'] + ', ' + subnet['id'])

Subnet: mwhicks212subnet, 8e4ec2eb-e65d-4cb1-b986-9e6b77e51e24


#### Add a Router

TODO: add links here

In [9]:
router = create_router(router_name, gw_network_name='public')
#print(json.dumps(router, indent=2))

if VERBOSE:
    print(json.dumps(router, indent=2))
else:
    print('Router: ' + router['name'] + ', ' + router['id'])

Router: mwhicks212Router, 3e5031d5-331c-4a77-b579-7856c55803f2


#### Attach the Router and Subnet

TODO: Add links here

In [10]:
add_subnet_to_router(get_router_id(router_name), get_subnet_id(subnet_name))

{'network_id': 'ea9236b1-032a-4990-8837-8e30f00cbebb',
 'tenant_id': 'ae76673270164b048b59d3bd30676721',
 'subnet_id': '8e4ec2eb-e65d-4cb1-b986-9e6b77e51e24',
 'subnet_ids': ['8e4ec2eb-e65d-4cb1-b986-9e6b77e51e24'],
 'port_id': '0f201af7-a205-4a13-9925-588ef450bc2e',
 'id': '3e5031d5-331c-4a77-b579-7856c55803f2'}

## Create TACC Network and Server

#### Create a Lease at Chicago

In [11]:
node_type="compute_cascadelake"

#Set the region
chi.use_site('CHI@TACC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


{'created_at': '2021-04-28 16:04:14',
 'updated_at': '2021-04-28 16:04:24',
 'id': '05952c0a-dbc8-4cb0-aad3-1945c3cf0696',
 'name': 'mwhicks212Lease',
 'user_id': 'bb8689c345c1b4ca36a8ef7c0898b828722386b625db3f26c28d0ddbc3ace92d',
 'project_id': 'a400724e818d40cbba1a5c6b5e714462',
 'start_date': '2021-04-28T16:05:00.000000',
 'end_date': '2021-04-29T16:04:00.000000',
 'trust_id': '66a226b454dd4b7293c81c553280e43c',
 'status': 'PENDING',
 'degraded': False,
 'reservations': [{'created_at': '2021-04-28 16:04:14',
   'updated_at': '2021-04-28 16:04:23',
   'id': '4ca4d249-579a-4afd-9206-b9135ca31d5c',
   'lease_id': '05952c0a-dbc8-4cb0-aad3-1945c3cf0696',
   'resource_id': '142ba35f-d03d-45a8-8f18-e8d8f8c00e0b',
   'resource_type': 'physical:host',
   'status': 'pending',
   'missing_resources': False,
   'resources_changed': False,
   'hypervisor_properties': '',
   'resource_properties': '["==", "$node_type", "compute_cascadelake"]',
   'before_end': 'default',
   'on_start': 'default',

In [12]:
chi.use_site('CHI@TACC')

#Get the lease by name
tacc_lease = get_lease(get_lease_id(lease_name))
    
#Print the lease info
if VERBOSE:
    print(json.dumps(tacc_lease, indent=2))
else:
    print('tacc_lease: ' + tacc_lease['name'] + ', ' + tacc_lease['id'])

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org
tacc_lease: mwhicks212Lease, 05952c0a-dbc8-4cb0-aad3-1945c3cf0696


#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [13]:
#Get the lease by name
tacc_lease = get_lease(get_lease_id(lease_name))

tacc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', tacc_lease['reservations']))[0]['id']
tacc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', tacc_lease['reservations']))[0]['id']
tacc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', tacc_lease['reservations']))[0]['id']

print("tacc_compute_reservation_id: " + tacc_compute_reservation_id)
print("tacc_network_reservation_id: " + tacc_network_reservation_id)
print("tacc_floatingip_reservation_id: " + tacc_floatingip_reservation_id)

tacc_compute_reservation_id: 4ca4d249-579a-4afd-9206-b9135ca31d5c
tacc_network_reservation_id: c00abb03-034e-432f-9bb5-6c684d1ebdcc
tacc_floatingip_reservation_id: cf1c24a6-65ce-4f1f-a111-64fe130452ec


#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [15]:
chi.use_site('CHI@TACC')    # Optional, defaults to 'CHI@UC'

#Get the network
tacc_network = get_network(get_network_id(network_name))
tacc_network_id = tacc_network['id']
tacc_network_vlan = tacc_network['provider:segmentation_id']

if VERBOSE:
    print(json.dumps(tacc_network, indent=2))
else:
    print('tacc_network: ' + tacc_network['name'] + ', ' + tacc_network['id'])
    print('tacc segmenation ID/VLAN: ' + str(tacc_network_vlan))

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org
tacc_network: mwhicks212Net, 43047100-fd19-4474-8724-cf3ca009daf3
tacc segmenation ID/VLAN: 3509


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [16]:
subnet = create_subnet(subnet_name, 
                       tacc_network_id, 
                       cidr=cidr, 
                      allocation_pool_start=tacc_allocation_start,
                      allocation_pool_end=tacc_allocation_end,
                      gateway_ip=tacc_gateway)

 

#print(json.dumps(subnet, indent=2))

In [17]:
if VERBOSE:
    print(json.dumps(subnet, indent=2))
else:
    print('Subnet: ' + subnet['name'] + ', ' + subnet['id'])

Subnet: mwhicks212subnet, 0eb92414-0a54-41e9-9384-2fcd8ea2b984


#### Add a Router

TODO: add links here

In [18]:
router = create_router(router_name, gw_network_name='public')
#print(json.dumps(router, indent=2))

if VERBOSE:
    print(json.dumps(router, indent=2))
else:
    print('Router: ' + router['name'] + ', ' + router['id'])

Router: mwhicks212Router, 9a1fb55f-c4b9-4069-8c5e-2a9c587ba10c


#### Attach the Router and Subnet

TODO: Add links here

In [19]:
add_subnet_to_router(get_router_id(router_name), get_subnet_id(subnet_name))

{'network_id': '43047100-fd19-4474-8724-cf3ca009daf3',
 'tenant_id': 'a400724e818d40cbba1a5c6b5e714462',
 'subnet_id': '0eb92414-0a54-41e9-9384-2fcd8ea2b984',
 'subnet_ids': ['0eb92414-0a54-41e9-9384-2fcd8ea2b984'],
 'port_id': 'f3d64b53-d973-4b77-9c69-bb0f432bd54d',
 'id': '9a1fb55f-c4b9-4069-8c5e-2a9c587ba10c'}

## Start the Servers

#### Start the UC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [20]:
chi.use_site('CHI@UC')  
#create the server
server = create_server(server_name, 
                       reservation_id=uc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [23]:
chi.use_site('CHI@UC')
server_id = get_server_id(server_name)
uc_floating_ip = associate_floating_ip(server_id)

uc_fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]

In [27]:
print('Floating IP: ' + str(uc_floating_ip))
print('Fixed IP: ' + str(uc_fixed_ip))

Floating IP: 192.5.87.206
Fixed IP: 192.168.42.140


#### Start the TACC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [24]:
chi.use_site('CHI@TACC')

print("server_name " + server_name)
print("tacc_compute_reservation_id " + tacc_compute_reservation_id)
print("key_name " + key_name) 
print("network_name " + network_name)
print("image_name " + image_name)
print("flavor_name " + flavor_name)

#create the server
server = create_server(server_name, 
                       reservation_id=tacc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org
server_name mwhicks212Server
tacc_compute_reservation_id 4ca4d249-579a-4afd-9206-b9135ca31d5c
key_name mac_chameleon_key
network_name mwhicks212Net
image_name CC-CentOS7
flavor_name baremetal


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [25]:
chi.use_site('CHI@TACC')  

server_id = get_server_id(server_name)
tacc_floating_ip = associate_floating_ip(server_id)
tacc_fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]

In [26]:
print('Floating IP: ' + str(tacc_floating_ip))
print('Fixed IP: ' + str(tacc_fixed_ip))

Floating IP: 129.114.108.43
Fixed IP: 192.168.42.219


## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

#### Check the Status of the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

## Access the Circuit

#### Create scripts

In [38]:
import paramiko

uc_script = '#!/bin/bash'   '\n' \
    'yum install vim iperf3 -y'   '\n' \
    'iperf3 -s'   '\n'
tacc_script = '#!/bin/bash'   '\n' \
    '{'   '\n' \
    'yum install vim iperf3 -y'   '\n' \
    'iperf3 -t 30 -i 5 -c '+uc_fixed_ip+   '\n' \
    '} > /tmp/boot.log 2>&1'   '\n' 

#### Wait TCP Accessible

In [39]:
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

# The time it takes for your server to be TCP-ready can vary greatly.
# This function, chi.server.wait_for_tcp, halts your cell's progress until the IP is tcp-accessible on the SSH port.
# The timeout is set to 45 minutes (60*45 seconds) due to greatly varying start-up times.
wait_for_tcp(uc_floating_ip, '22', timeout=(60*45))
wait_for_tcp(uc_floating_ip, '22', timeout=(60*45))
print('done')

uc_key_location = "/home/mwhicks2/work/mac_chameleon_key_uc.pem"
tacc_key_location = "/home/mwhicks2/work/mac_chameleon_key_tacc.pem"

uc_key = paramiko.RSAKey.from_private_key_file(uc_key_location)
tacc_key = paramiko.RSAKey.from_private_key_file(tacc_key_location)

done


#### Run UC Script

In [43]:
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(uc_floating_ip,username='cc',pkey = uc_key) # Sometimes this line may throw an error. Wait a few minutes and then try again!

stdin, stdout, stderr = client.exec_command('echo \"' + uc_script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')

(<paramiko.ChannelFile from <paramiko.Channel 0 (open) window=2097152 -> <paramiko.Transport at 0x1b2ac6a0 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 0 (open) window=2097152 -> <paramiko.Transport at 0x1b2ac6a0 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 0 (open) window=2097152 -> <paramiko.Transport at 0x1b2ac6a0 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>)

#### Run TACC Script

In [44]:
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(tacc_floating_ip,username='cc',pkey = tacc_key) # Sometimes this line may throw an error. Wait a few minutes and then try again!

stdin, stdout, stderr = client.exec_command('echo \"' + tacc_script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')
print (stdout.read())
print (stderr.read())

b''
b''


## Clean Up Resources

#### Close Paramiko Client

In [ ]:
client.close()

### Delete Stitched Circuit using ExoGENI

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

### Delete TACC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
chi.use_site('CHI@TACC')

In [ ]:
delete_server(get_server_id(server_name))

#### De-configure Network
TODO: break up into steps

In [ ]:
remove_subnet_from_router(router['id'], subnet['id'])

In [ ]:
delete_router(router['id'])

In [ ]:
delete_subnet(subnet['id'])

In [ ]:
delete_network(tacc_network['id'])

#### Release Lease

In [ ]:
delete_lease(tacc_lease['id'])

### Delete UC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
chi.use_site('CHI@UC')

In [ ]:
delete_server(get_server_id(server_name))

#### De-configure Network
TODO: break up into steps

In [ ]:
remove_subnet_from_router(get_router_id(router_name), get_subnet_id(subnet_name))

In [ ]:
delete_router(get_router_id(router_name))

In [ ]:
delete_subnet(get_subnet_id(subnet_name))

In [ ]:
delete_network(uc_network['id'])

#### Release Lease

In [ ]:
delete_lease(uc_lease['id'])